# Property Analytics

[README.md](README.md)

***

## Scope of work
- Business context
  - Problem statement
- Exploratory data analysis
- Solution
  - Technologies and tools
  - Architecture and design
  - Implementation
    - Setup analytics environment
    - Implement Extract features
    - Implement Load features
    - Implement Transform features
  - Analytics
    - Cost per visit
    - Qualified properties and associated costs
    - Conversions and costs of marketing campaigns
    - Data model improvements
  - Evolution
    - Architecture
    - Query optimization
    - Change management
***

## Business Context
In a property business, the following processes are running:
- Marketing campaign management
- Website visits tracking
- Property inventory management
- Property events logging

Business stakeholders need:
 - Insights related to:
    - Marketing costs per visit
    - Qualified properties (i.e. those which passed validation by property visitors) and associated costs
    - Conversions and marketing campaign costs
 - Recommendations on data model improvements

***

## Exploratory Data Analysis
As the first step, let us perform exploratory data analysis (EDA) to get general understanding of the data available,
 its quality, and basic statistics.

In [1]:
import pandas as pd

In [2]:
SRC_DATA_PATH = 'src_data'

GA_DATA = 'google_analytics'
GA_FILENAME = 'ga_sessions_20190106'

MARKETING_DATA = 'google_sheets'
COSTS_FILENAME = 'manual_marketing_cost_input'

CRM_DATA = 'mysql'
PROPERTIES_FILENAME = 'properties'
PROPERTY_EVENTS_FILENAME = 'property_event_log'

SRC_FILE_EXT = 'csv'

### Marketing costs

In [3]:
marketing_costs = pd.read_csv(f'{SRC_DATA_PATH}/{MARKETING_DATA}/{COSTS_FILENAME}.{SRC_FILE_EXT}')
marketing_costs.shape

(3742, 9)

In [4]:
marketing_costs.head(2)

,From,To,Cost Type,Medium,Source,Campaign,Term,Content,Cost
0,01.01.19,01.01.19,fixed,SEO,NaN,NaN,NaN,NaN,101.1000
1,01.01.19,01.01.19,variable,display,taboola,HD_TB_DE_RH_DT_SLIDER_LOHNT-VERKAUF,NaN,NaN,1741.4912


In [5]:
marketing_costs.describe(include = 'all')

,From,To,Cost Type,Medium,Source,Campaign,Term,Content,Cost
count,3742,3742,3742,3742,3738,3698,3036,2.981000e+03,3742.000000
unique,31,32,2,7,24,77,94,NaN,NaN
top,30.01.19,31.01.19,variable,display,dv,7132134,23105367,NaN,NaN
freq,163,169,3736,3165,2981,321,220,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.561807e+08,562.366575
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.549032e+06,2479.806565
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.537618e+08,1.100000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.557924e+08,91.437500
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.558153e+08,238.180000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.560039e+08,601.785000


The Marketing Costs dataset contains information about marketing campaigns along with the dates, type, medium,
source, and costs.
Marketing campaigns started every day in January 2019, and most of them were running for a period of less than two
weeks.

### Google Analytics data

In [6]:
ga_data = pd.read_csv(f'{SRC_DATA_PATH}/{GA_DATA}/{GA_FILENAME}.{SRC_FILE_EXT}')
ga_data.shape

(15153, 10)

In [7]:
ga_data.head(2)

,FILE,VISIT_ID,VISIT_START_TIME::TIMESTAMP_TZ,DATE,TOTALS,TRAFFIC_SOURCE,DEVICE,CUSTOM_DIMENSIONS,HITS,CLIENT_ID
0,ga_sessions_180842477_20190106.json,1546766921,2019-01-06T09:28:41Z,20190106,"{""hits"":""1"",""sessionQualityDim"":""0""}","{""adContent"":""(not set)"",""adwordsClickInfo"":{}...","{""browser"":""Safari"",""browserSize"":""380x640"",""b...",[],"[{""appInfo"":{""exitScreenName"":""www.homeday.de/...",1374893137
1,ga_sessions_180842477_20190106.json,1546807837,2019-01-06T20:50:37Z,20190106,"{""hits"":""1"",""sessionQualityDim"":""0""}","{""adContent"":""(not set)"",""adwordsClickInfo"":{}...","{""browser"":""Safari"",""browserSize"":""1020x670"",""...",[],"[{""appInfo"":{""exitScreenName"":""www.homeday.de/...",2051673004


In [8]:
ga_data.describe(include = 'all')

,FILE,VISIT_ID,VISIT_START_TIME::TIMESTAMP_TZ,DATE,TOTALS,TRAFFIC_SOURCE,DEVICE,CUSTOM_DIMENSIONS,HITS,CLIENT_ID
count,15153,1.515300e+04,15153,15153.0,15153,15153,15153,15153,15153,1.515300e+04
unique,1,NaN,13353,NaN,3569,1482,7182,839,14695,NaN
top,ga_sessions_180842477_20190106.json,NaN,2019-01-06T16:44:26Z,NaN,"{""bounces"":""1"",""hits"":""5"",""newVisits"":""1"",""pag...","{""adContent"":""(not set)"",""adwordsClickInfo"":{}...","{""browser"":""Safari"",""browserSize"":""380x550"",""b...",[],"[{""appInfo"":{""exitScreenName"":""www.homeday.de/...",NaN
freq,15153,NaN,6,NaN,1978,1795,303,3990,6,NaN
mean,NaN,1.546781e+09,NaN,20190106.0,NaN,NaN,NaN,NaN,NaN,1.081214e+09
std,NaN,1.791087e+04,NaN,0.0,NaN,NaN,NaN,NaN,NaN,6.226834e+08
min,NaN,1.546728e+09,NaN,20190106.0,NaN,NaN,NaN,NaN,NaN,2.443160e+05
25%,NaN,1.546769e+09,NaN,20190106.0,NaN,NaN,NaN,NaN,NaN,5.435725e+08
50%,NaN,1.546779e+09,NaN,20190106.0,NaN,NaN,NaN,NaN,NaN,1.092500e+09
75%,NaN,1.546794e+09,NaN,20190106.0,NaN,NaN,NaN,NaN,NaN,1.618156e+09


Dataset exported from Google Analytics contains data about site visits on 2019-01-06. Each visit is
marked with a unique timestamp, `visit_id`, and semi-structured data on totals, traffic sources, devices, and hits.
For each visit, the dataset also provides `client_id` (not unique).

### Properties inventory

In [9]:
properties = pd.read_csv(f'{SRC_DATA_PATH}/{CRM_DATA}/{PROPERTIES_FILENAME}.{SRC_FILE_EXT}')
properties.shape

(446, 22)

In [10]:
properties.head(2)

,ID,ZIP_CODE,APARTMENT_TYPE,LIVING_SPACE,CITY,CREATED_AT,UID,UPDATED_AT,CONSIDERS_TO_SELL,SLIDER_TYPE,...,STAGE,COUNTRY,UTM_MEDIUM,MARKETING_TYPE,LOSS_REASON_ID,ADMIN_USER_ID,UTM_CAMPAIGN,UTM_SOURCE,BUILDING_TYPE,LANDING_PAGE
0,268541,86836.0,NaN,125.0,Graben,2019-01-09 01:15:19.000 +0000,I0VOLDR7,2019-06-11 16:51:50.000 +0000,asap,sale,...,lost,DE,display,sell,26.0,139.0,HD_PL_DE_RH_DT_SLIDER_LOHNT-VERKAUF,plista,semidetached_house,https://www.homeday.de/de/di/immobilienverkauf
1,268545,53819.0,apartment,55.0,Lo,2019-01-09 05:20:36.000 +0000,NYDZUIOF,2019-07-25 03:11:01.000 +0000,not_owner,valuation,...,property_contact,DE,NaN,sell,NaN,NaN,NaN,NaN,NaN,www.homeday.de/de/preisatlas/lohmar/siegburger...


In [11]:
properties.describe(include = 'all')

,ID,ZIP_CODE,APARTMENT_TYPE,LIVING_SPACE,CITY,CREATED_AT,UID,UPDATED_AT,CONSIDERS_TO_SELL,SLIDER_TYPE,...,STAGE,COUNTRY,UTM_MEDIUM,MARKETING_TYPE,LOSS_REASON_ID,ADMIN_USER_ID,UTM_CAMPAIGN,UTM_SOURCE,BUILDING_TYPE,LANDING_PAGE
count,446.000000,443.000000,125,413.000000,446,446,446,446,383,384,...,446,446,421,446,212.000000,248.000000,419,420,194,364
unique,NaN,NaN,4,NaN,256,446,446,346,8,4,...,7,1,8,2,NaN,NaN,88,17,6,113
top,NaN,NaN,apartment,NaN,[ask in call],2019-01-09 01:15:19.000 +0000,I0VOLDR7,2019-08-28 03:05:32.000 +0000,no,sale,...,property_contact,DE,display,sell,NaN,NaN,immowelt,immowelt,single_family_house,www.homeday.de/de/di/immobilienverkauf
freq,NaN,NaN,71,NaN,134,1,1,7,87,174,...,217,446,169,430,NaN,NaN,110,120,85,96
mean,268762.500000,53371.108352,NaN,142.050412,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,29.415094,114.431452,NaN,NaN,NaN,NaN
std,128.893367,33574.157927,NaN,100.014269,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,12.597576,58.254184,NaN,NaN,NaN,NaN
min,268540.000000,80.000000,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,23.000000,8.000000,NaN,NaN,NaN,NaN
25%,268651.250000,28517.500000,NaN,85.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,24.000000,89.000000,NaN,NaN,NaN,NaN
50%,268762.500000,56204.000000,NaN,120.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,26.000000,130.000000,NaN,NaN,NaN,NaN
75%,268873.750000,75273.500000,NaN,165.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,29.000000,164.000000,NaN,NaN,NaN,NaN


The Properties dataset contains an inventory of converted properties (i.e. those selected by visitors), with
a unique `id`, property features, `created_at` and `updated_at` timestamps, and the stage of the sales process.

### Property events log

In [12]:
property_events = pd.read_csv(f'{SRC_DATA_PATH}/{CRM_DATA}/{PROPERTY_EVENTS_FILENAME}.{SRC_FILE_EXT}')
property_events.shape

(3243, 6)

In [13]:
property_events.head(2)

,ID,UPDATED_AT,OBJECT_TYPE,ACTION,CREATED_AT,OBJECT_ID
0,7402455,2019-01-09 10:36:49.000 +0000,Property,PropertyContactCreated,2019-01-09 10:36:49.000 +0000,268646
1,7402458,2019-01-09 10:36:49.000 +0000,Property,PropertyStagePropertyContact,2019-01-09 10:36:49.000 +0000,268646


In [14]:
property_events.describe(include = 'all')

,ID,UPDATED_AT,OBJECT_TYPE,ACTION,CREATED_AT,OBJECT_ID
count,3.243000e+03,3243,3243,3243,3243,3243.000000
unique,NaN,2176,1,39,2176,NaN
top,NaN,2019-01-10 17:00:15.000 +0000,Property,Update,2019-01-10 17:00:15.000 +0000,NaN
freq,NaN,10,3243,575,10,NaN
mean,7.698977e+06,NaN,NaN,NaN,NaN,268756.896392
std,1.329680e+06,NaN,NaN,NaN,NaN,130.873473
min,7.398236e+06,NaN,NaN,NaN,NaN,268540.000000
25%,7.403832e+06,NaN,NaN,NaN,NaN,268640.000000
50%,7.409128e+06,NaN,NaN,NaN,NaN,268760.000000
75%,7.421546e+06,NaN,NaN,NaN,NaN,268871.000000


The Property Events dataset contains the log of property changes from a CRM application for a property object
(`object_id`), along with the `created_at` and `updated_at` timestamps (the timestamps are equal across the dataset).

***

## Solution

### Technologies and Tools

The solution is based on open-source technologies, with Python and SQL used for the implementation of the ELT data
pipeline, and PostgreSQL used as the data warehouse.

The tools used are IDEs for Python and SQL development, and Unix CLI.

### Architecture: Prototype

<img src="./docs/analytics_architecture__as_is.png" alt="architecture" width="800"/>

#### Design

The following design principles are applied:
- Separation of logic (Infra, ELT, Analytics)
- Idempotency of operations
- Configuration-driven runs

### Implementation
The implementation of the solution includes:
- Setting up the environment
    - Database
    - Schemas
    - Tables
- Implementation of the ELT pipeline
    - Extract (already implemented)
    - Load
    - Transform
- Implementation of the analytics
    - Data marts

#### Setup environment

Environment setup includes the creation of local PostgreSQL database `propertydb`.

In [15]:
import sys
sys.path.insert(0, './config')

import config.db_utils as db_utils

In [16]:
# Create a new PostgreSQL database `propertydb` (if not exists)
!./config/db_init.sh

Initializing the Property Analytics solution...
Creating database...
Done.


In [17]:
# Test database connection
db_utils.test_connection()

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 14.5 (Homebrew) on x86_64-apple-darwin20.6.0, compiled by Apple clang version 13.0.0 (clang-1300.0.29.30), 64-bit',)
Closing database connection... 
Done.


#### Extract Data

Data Extraction and Load are performed by exporting daily batches from data sources (Google Sheets, Google Analytics,
 MySQL) to a Data Lake (e.g. Google Cloud Storage) as CSV files.

 In this project, data extraction services are already implemented, and we will load data from local CSV files to the
  DWH (PostgreSQL in this case).


#### Load Data

To load data into the DWH, we will use schema `staging` and tables:
- `staging.ga_sessions`
- `staging.marketing_costs`
- `staging.properties`
- `staging.property_events`

In [18]:
sys.path.insert(1, './elt')

import elt.db_schema as schema
import elt.load as load

In [19]:
schema.create_schema(schema_name='staging')

Creating schema `staging`...
Done.


In [20]:
ddl_scripts = (
    'elt/ddl/table__staging_marketing_costs.sql',
    'elt/ddl/table__staging_ga_sessions.sql',
    'elt/ddl/table__staging_properties.sql',
    'elt/ddl/table__staging_property_events.sql'
)

schema.create_tables(ddl_scripts=ddl_scripts)

Creating tables...
Done.


In [21]:
costs_file_path = f'{SRC_DATA_PATH}/{MARKETING_DATA}/{COSTS_FILENAME}.{SRC_FILE_EXT}'

load.file_to_table(file_path=costs_file_path,
                   schema='staging',
                   table='marketing_costs',
                   chunk=500)

Connecting to the database...
Successfully connected to propertydb.
Loading batch to table staging.marketing_costs...
Done.


In [22]:
ga_file_path = f'{SRC_DATA_PATH}/{GA_DATA}/{GA_FILENAME}.{SRC_FILE_EXT}'

load.file_to_table(file_path=ga_file_path,
                   schema='staging',
                   table='ga_sessions',
                   chunk=1000)

Connecting to the database...
Successfully connected to propertydb.
Loading batch to table staging.ga_sessions...
Done.


In [23]:
properties_file_path = f'{SRC_DATA_PATH}/{CRM_DATA}/{PROPERTIES_FILENAME}.{SRC_FILE_EXT}'

load.file_to_table(file_path=properties_file_path,
                   schema='staging',
                   table='properties',
                   chunk=100)

Connecting to the database...
Successfully connected to propertydb.
Loading batch to table staging.properties...
Done.


In [24]:
property_events_file_path = f'{SRC_DATA_PATH}/{CRM_DATA}/{PROPERTY_EVENTS_FILENAME}.{SRC_FILE_EXT}'

load.file_to_table(file_path=property_events_file_path,
                   schema='staging',
                   table='property_events',
                   chunk=500)

Connecting to the database...
Successfully connected to propertydb.
Loading batch to table staging.property_events...
Done.


#### Transform Data

Data transformation is performed in schema `analytics`.

The result data marts are implemented as views:
- `analytics.periods_daily_costs`: provides the marketing costs data transformed into periods and daily costs for
each period
- `analytics.daily_visits`: provides counts of visits by day
- `analytics.properties`: provides a list of properties with their conversion dates, qualification dates, and
associated campaigns
- `analytics.campaigns_costs`: provides total costs for each campaign

In [25]:
import elt.transform as transform

In [26]:
schema.create_schema(schema_name='analytics')

Creating schema `analytics`...
Done.


In [27]:
ddl_scripts = (
    "elt/ddl/view__analytics_periods_daily_costs.sql",
    "elt/ddl/view__analytics_daily_visits.sql",
    "elt/ddl/view__analytics_properties.sql",
    "elt/ddl/view__analytics_campaigns_costs.sql"
)

transform.create_marts(ddl_scripts=ddl_scripts)

Creating data marts...
Done.


***

### Analytics

Analytics consists in querying the data marts and loading the result datasets as Pandas dataframes.

Each query below is intended to provide an example of one of the ways to answer the questions from the
business.

#### Cost per visit

This section provides the queries which can help answer the following question:
- *What is the cost per visit on a given day?*

In [28]:
from elt.db import connect

In [29]:
engine = connect()

Connecting to the database...
Successfully connected to propertydb.


In [30]:
query_cost_per_visit = \
    """
    WITH
        daily_visits_costs AS (
            SELECT
                v.visit_date                                        AS visit_date,
                v.visits_per_day                                    AS visits_per_day,
                v.unique_visits_per_day                             AS unique_visits_per_day,
                c.period_daily_cost                                 AS period_daily_cost,
                c.period_daily_cost / v.visits_per_day              AS period_cost_per_visit,
                c.period_daily_cost / v.unique_visits_per_day       AS period_cost_per_unique_visit
            FROM
                analytics.daily_visits v
                LEFT JOIN analytics.periods_daily_costs c
                    ON v.visit_date BETWEEN c.date_from AND c.date_to
        )
    SELECT
        visit_date                                                  AS visit_date,
        sum(period_cost_per_visit)                                  AS cost_per_visit,
        sum(period_cost_per_unique_visit)                           AS cost_per_unique_visit
    FROM
        daily_visits_costs
    GROUP BY 1;
    """

cost_per_visit = pd.read_sql_query(query_cost_per_visit, con=engine)
cost_per_visit.head()

,visit_date,cost_per_visit,cost_per_unique_visit
0,2019-01-06,0.416092,0.472111


#### Qualified properties and associated costs

This section provides the queries which can help answer the following question:
- *How many properties are qualified and what is the cost per qualified property on a given day?*

In [31]:
query_cost_per_qualified_property = \
    """
    WITH
        properties_qualified AS (
            WITH
                properties_qualified_date AS (
                    SELECT
                        property_id                                 AS property_id,
                        date(qualified_at)                          AS date_qualified
                    FROM
                        analytics.properties
                    WHERE
                        qualified_at IS NOT NULL -- only qualified
                )
            SELECT
                date_qualified                                      AS date_qualified,
                count(property_id)                                  AS properties_cnt
            FROM
                properties_qualified_date
            GROUP BY 1
        ),
        daily_qualified_properties_costs AS (
            SELECT
                pq.date_qualified                                   AS date_qualified,
                pq.properties_cnt                                   AS properties_cnt,
                c.period_daily_cost                                 AS period_daily_cost,
                c.period_daily_cost / pq.properties_cnt             AS period_cost_per_qualified_property
            FROM
                properties_qualified pq
                LEFT JOIN analytics.periods_daily_costs c
                    ON pq.date_qualified BETWEEN c.date_from AND c.date_to
        )
    SELECT
        date_qualified                                              AS date_qualified,
        sum(period_cost_per_qualified_property)                     AS cost_per_qualified_property
    FROM
        daily_qualified_properties_costs
    GROUP BY 1
    ORDER BY 1;
    """

cost_per_qualified_property = pd.read_sql_query(query_cost_per_qualified_property, con=engine)
cost_per_qualified_property.head(5)

,date_qualified,cost_per_qualified_property
0,2019-01-09,116.241350
1,2019-01-10,639.380476
2,2019-01-11,1459.193165
3,2019-01-12,7229.466424
4,2019-01-22,2691.385754


#### Conversions and costs of marketing campaigns

This section provides the queries which can help answer the following question:
- *How many conversions are acquired and what is the cost of a given marketing campaign?*

In [32]:
query_campaign_conversions_costs = \
    """
    WITH
        properties_converted AS (
            WITH
                properties_converted_campaign AS (
                    SELECT
                        property_id                                 AS property_id,
                        campaign                                    AS campaign
                    FROM
                        analytics.properties
                )
            SELECT
                campaign                                            AS campaign,
                count(property_id)                                  AS properties_cnt
            FROM
                properties_converted_campaign
            GROUP BY 1
        )
    SELECT
        pc.campaign                                                 AS campaign,
        pc.properties_cnt                                           AS converted_properties,
        cc.total_cost                                               AS campaign_cost
    FROM
        properties_converted pc
        LEFT JOIN analytics.campaigns_costs cc
            ON pc.campaign = cc.campaign
    WHERE
        cc.total_cost IS NOT NULL
    ORDER BY
        cc.total_cost DESC;
    """

campaign_conversions_costs = pd.read_sql_query(query_campaign_conversions_costs, con=engine)
campaign_conversions_costs.head(10)

,campaign,converted_properties,campaign_cost
0,HD_TB_DE_RH_DT_SLIDER_LOHNT-VERKAUF,100,58152.3468
1,Immo_Verkauf_2.0,7,25304.2000
2,HD_PL_DE_RH_DT_SLIDER_LOHNT-VERKAUF,17,24470.8000
3,HD_PL_RH_MB_CONTENT_TODSUENDEN,3,15992.4000
4,HD_TB_DE_RH_DT_APP-SLIDER_LOHNT-VERKAUF,3,13584.0696
5,HD_OB_DE_RH_DT_APP-SLIDER_LOHNT-VERKAUF,1,13449.9400
6,HD_OB_DE_RT_DT_SLIDER_LOHNT-VERKAUF,4,12684.8100
7,HD_PL_DE_RH_DT_CONTENT_TODSUENDEN,11,12469.9000
8,HD_TB_DE_RH_DT_SLIDER_PLZ-EINGABE,1,11932.3208
9,Maklerprovision_1.0,2,9503.6000


The biggest issue with the data is its quality.
For example, for many marketing costs, there is no campaign specified. To deal with this issue, each such cost is
marked as a `default_campaign`.
Another example is missing campaign in the `properties` dataset. This is a way how the producer of this data could
improve it for analytics.

## Evolution
This project can be viewed as a prototype of a production solution. The evolved solution would require the following roles:
- Data Engineer
- Analytics Engineer
- Data Analyst / BI Developer

From the solution engineering viewpoint, the following concepts of Modern Data Platform should be taken into
condideration:
- Cloud DWH, Data Lake, Data LakeHouse
    - Cloud DWH
        - Redshift
        - BigQuery
        - Snowflake
    - Data Lake
        - S3
        - Google Cloud Storage
    - Data LakeHouse
        - Databricks
- Data modeling
    - Snowflake
    - Star schema
    - Data Vault
    - Anchor modeling
    - Activity Schema
- ELT tools
    - Job orchestration and scheduling, e.g. Airflow
    - MPP, e.g. Spark
    - Modern data transformation tool, e.g. dbt
- DataViz tools
    - BI tools of choice, e.g. Tableau, Superset, Metabase, Google Looker Studio
- DevOps
    - Containerization for deployments, e.g. Docker
    - CI/CD solution, e.g. git + Docker + k8s + Jenkins
- AI stack
    - ML tasks
    - ML data pipelines and MLOps
- Data Observability
    - Monitoring and Logging
        - Specialized toolset, e.g. ELK stack
    - Data Quality
        - Modern DQ tools, e.g. great_expectations
    - Data Catalog
        - A data catalog solution, e.g. dbt + DataHub
    - FinOps
        - Monitoring and optimization of cloud costs

### Architecture

<img src="./docs/analytics_architecture__to_be.png" alt="architecture" width="800"/>

A few points on the suggested architecture:
- A cloud object storage (**Amazon S3**, **Google Cloud Storage**) can be used as the Data Lake. This will enable
distributed data storage with bucket access and retention configuration.
- **Apache Parquet** can be used a the data format for better performance, especially for large datasets.
- **Apache Airflow** can be used as the orchestrator and scheduler of ELT data pipelines. This will improve further
implementation and maintenance of data extraction, load, and transformation flows.
- **Snowflake** can be used as the cloud-based DWH built on top of the data files stored in a cloud storage.
- **dbt** can be used for SQL-based data transformations. It will enable applying modern software engineering
practices to data modeling, data transformations, analytics marts development, data cataloging.
- **VS Code** IDE currently provides the best support for dbt-driven development.
- **Tableau** is one of the most popular and feature-rich BI tool.

Potential disadvantages:
- Vendor lock (Google, Amazon, Snowflake)
- Cloud costs

***

### Query optimization
This section contains SQL query with performance issues observed.
A few comments are added to highlight problematic parts and ways to handle them.

In [33]:
query_not_so_optimal = \
    """
    WITH
        LEAD_EVENTS AS
        (
            SELECT DISTINCT
                UID AS TOUCHPOINT_ID
            FROM
                SSTAR.FACT.GA_EVENTS
            WHERE
                (IS_LEAD OR IS_PROPERTY_CONTACT OR IS_RESELL)
            GROUP BY UID  -- does not make sense here (DISTINCT, no aggregations), expensive operation
            HAVING UID IS NOT NULL  -- this condition can be added to the WHERE clause
        ),

        TOUCHPOINT_ATTRIBUTION AS
        (
            SELECT
	            PROPERTY_ID,
                CASE
                    WHEN UTM_SOURCE LIKE ‘%homeday%’  -- LIKE should be avoided in production, it's slow
                    THEN concat(UTM_SOURCE, TOUCHPOINT_ID)
                END AS HOMEDAY_SOURCE,
                TOUCHPOINT_ID,
                TOUCHPOINT_VALUE
            FROM
                MLOGIC.ATTR.PATH_TOUCHPOINTS
        )

    SELECT
        * -- not a good practice, brings ambiguity, makes debugging harder
    FROM
	( -- should be a CTE to follow the modularity principle and improve maintenance
	    SELECT
		    SE.TOUCHPOINT_ID,
			SE.TOUCHPOINT_DATE,
			COALESCE(AC.COST_VAR_PER_SESSION, 0)  + COALESCE(MC.COST_VAR_PER_SESSION, 0) AS COST_VAR_PER_SESSION,
			COALESCE(AC.COST_FIX_PER_SESSION, 0)  + COALESCE(MC.COST_FIX_PER_SESSION, 0) AS COST_FIX_PER_SESSION
		FROM
		    -- not a good practice, may result in cross joins
		    MLOGIC.ATTR.MARKETING_TOUCHPOINTS SE,
		    MLOGIC.COST.AD_COST_DISTRIBUTION AC,
		    MLOGIC.COST.SOURCE_MEDIUM_COST_DISTRIBUTION MC
		WHERE
		    SE.TOUCHPOINT_DATE = AC.TOUCHPOINT_DATE
			AND SE.AD_ID = AC.AD_ID
			AND SE.TOUCHPOINT_DATE = MC.TOUCHPOINT_DATE
			AND SE.MEDIUM_ID = MC.MEDIUM_ID
			AND SE.TOUCHPOINT_ID NOT IN
			    (SELECT * FROM LEAD_EVENTS ) -- * is a potential issue if the referenced CTE is extended
	) AS T  -- a semantically correct name would be better
    LEFT JOIN TOUCHPOINT_ATTRIBUTION TA
        ON TA.TOUCHPOINT_ID = T.TOUCHPOINT_ID
        AND T.TOUCHPOINT_DATE >= CURRENT_DATE -- replace with '=' ('>' is always True), move the filter into CTE
    WHERE
        TA.TOUCHPOINT_VALUE > 0.5 -- makes sense to pushdown the filter to the CTE
    """

***

### Change Management
This section describes an approach to planning the implementation of system features to support a new business case.

#### Business Context
According to the business needs, the following new cases should be supported:
- Property sold
- Property lost
- Commission received

Business stakeholders need:
 - Insights related to property revenue

#### Planning
1. Identify the sources of data and structure:
    - Sold properties: events with timestamps, property identifiers, selling price, commission received
    - Lost properties: events with timestamps, property identifiers, lost reasons
2. Extend ELT features
    - Extract: implement or update existing data extraction routines
    - Load: implement or update existing routines for loading data to:
        - `staging.property_events`
        - `staging.*` (new table in case of a new data source)
    - Transform: implement the logic of transforming the data
3. Update analytics data marts:
    - Extend `analytics.properties`: implement logic and add fields `sold_at`, `lost_at`, `lost_reason`
    - Implement `analytics.commissions`: implement logic and add fields `price`, `commission`
4. Deploy and test ELT in a sandbox environment on a sample of data
5. Deploy and test ELT in production environment
6. Update BI reports and dashboards
7. Update Data Catalog